### Universidad del Valle de Guatemala
### Modelado y Simulacion
#### MICRO PROYECTO 4
##### Juan Marroquin 19845
##### Carlos Raxtum 19721

import libraries

In [1]:
import numpy as np
from random import random , uniform
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy.integrate import quad
import random

#### Ejercicio 1
#### El triángulo de Sierpinski es uno de los fractales más famosos. Un fractal se puede construir a través de saltos aleatorios entre funciones determinísticas (juegos de caos). Considere las siguientes tres funciones:
* Point f1(Point p): return Point(p.x/2, p.y/2)
* Point f2(Point p): return Point(p.x/2 + 0.5, p.y/2)
* Point f3(Point p): return Point(p.x/2 + 0.25, p.y/2 + 0.5)
#### Estas funciones toman como parámetro el punto anterior en el fractal, y lo utilizan para generar el siguiente punto (paso determinístico).
#### La idea de los juegos del caos es simular una variable aleatoria X, para elegir la función a utilizar para calcular el siguiente punto. Noten que X debe ser discreta por tanto:
        P(X=x1) = p1, P(X=x2) = p2, P(X=x3) = p3, tal que p1 + p2 + p3 = 1

In [ ]:
#### Tasks:
* 1. Cree un programa que simule 100,000 veces X para elegir entre f1, f2 y f3, dibuje un triángulo de Sierpinski
* 2. Determine experimentalmente p1, p2, p3 que hacen su dibujo más denso